In [1]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import numpy as np
import random
import json
import urllib
import re
from selenium import webdriver

import html

In [3]:
def get_html(url):
    r = requests.get(url)
    return r.text

def get_book_info(url):
    html = get_html(url)
    id_paper = int(url.split('/')[5])
    soup = BeautifulSoup(html, 'html.parser')
    image_src = soup.find_all('div', class_='product__image')[0].findAll("img")[0]
    image = image_src.attrs['src']
    
    name = soup.find_all('h1', itemprop='name')[0].text
    author = soup.find_all('a', class_='link product__author')[0].text.strip()
    
    rating = soup.find_all('div', class_='rating')[0].find('span').text
    stars = rating.split(' ')[0]
    voters = rating.split(' (')[1].split(')')[0]
    
    # add try
    try:
        price = soup.find_all('div', class_='price')[0].text.strip().split(' ')[0]
    except:
        price = 'Не указана'
    
    data = soup.find_all('div', class_='product-prop')
    data_table = [[data[i].find('div', class_='product-prop__title').text.strip(),
                   data[i].find('div', class_='product-prop__value').text.strip()]
                  for i, _ in enumerate(data)]
    all_data = {'ID': id_paper, 'image': image, 'name': name,
                'author': author, 'stars': stars, 'voters': voters,
                'price': price, 'table': data_table}
    return all_data

def book_dataframe(data):
    df = pd.DataFrame(index=np.arange(1), columns=['ID карточки', 'ID товара:', 'ISBN', 'Автор', 'Возраст', 
                   'Год издания', 'Голоса', 'Издательство', 'Кол-во страниц',
                   'Название', 'Переводчик', 'Редактор', 'Рейтинг', 'Серия',
                   'Тип обложки', 'Тираж', 'Формат', 'Художник', 'Цена', 'Обложка'])
    df['ID карточки'] = data['ID']
    df['Обложка'] = data['image']
    df['Название'] = data['name']
    df['Автор'] = data['author']
    df['Рейтинг'] = data['stars']
    df['Голоса'] = data['voters']
    df['Цена'] = data['price']

    for item in data['table']:
        df[item[0]] = item[1]
    return df

def get_ids(url):
    ids = []
    with webdriver.Chrome('/home/Albert/Загрузки/chromedriver_linux64/chromedriver') as driver:
        driver.get(url)
        i = 2    
        length = driver.find_elements_by_class_name("count-result")
        count = length[0].find_elements_by_class_name('count-result__value')[0].text

        items = driver.find_elements_by_class_name("product-card")
        while (len(items) != int(count)):
            driver.execute_script("window.scrollTo(0, {0}000);".format(i))
            sleep(4)
            i += 2
            driver.execute_script("window.scrollTo(0, {0}000);".format(i))
            sleep(4)
            try:
                play = driver.find_element_by_class_name('js__show-more-cards')
                play.click()
            except:
                pass
            sleep(5)
            items = driver.find_elements_by_class_name("product-card")
            i *= 4
            driver.execute_script("window.scrollTo(0, {0}000);".format(i))
            sleep(4)
            i += 2
            driver.execute_script("window.scrollTo(0, {0}000);".format(i))

        items = driver.find_elements_by_class_name("product-card")
        for item in items:
             ids.append(item.get_attribute('data-product'))
                
    return ids

def all_book_urls(ids):
    books_urls = ['https://www.chitai-gorod.ru/catalog/book/' + x 
                  + '/?watch_fromlist=search_result' for x in ids]
    return books_urls

def get_all_books(url):
    ids = get_ids(url)
    books_urls = all_book_urls(ids)
    
    data = pd.DataFrame()
    df = pd.DataFrame()

    for url in books_urls:
        df = book_dataframe(get_book_info(url))
        frame = [data, df]
        data = pd.concat(frame)
    return data

In [4]:
m_fry = urllib.parse.quote("Фрай М.", encoding="cp1251")
url = ('https://www.chitai-gorod.ru/search/result.php?q={0}&type=author').format(m_fry)
df1 = pd.DataFrame()
df1 = get_all_books(url)

In [5]:
e_hanter = urllib.parse.quote("Хантер Э.", encoding="cp1251")
url = ('https://www.chitai-gorod.ru/search/result.php?q={0}&type=author').format(e_hanter)
df2 = pd.DataFrame()
df2 = get_all_books(url)

In [6]:
d_emec = urllib.parse.quote("Емец Д.", encoding="cp1251")
url = ('https://www.chitai-gorod.ru/search/result.php?q={0}&type=author').format(d_emec)
df3 = pd.DataFrame()
df3 = get_all_books(url)

In [7]:
df0 = pd.DataFrame()
frame = [df1, df2, df3]
df0 = pd.concat(frame)
df0.index = range(441)

df0.to_csv('hw_4.csv')